# Project setup

In [1]:
!pip install h2o -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.8/265.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# Import libraries
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML

In [3]:
# Print version numbers of the libraries as well as the environment
print(f"System platform {sys.platform}")
print(f"Python version {sys.version}")
print("-------------------------------")
print(f"Pandas version {pd.__version__}")
print(f"Numpy version {np.__version__}")
print(f"Matplotlib version {matplotlib.__version__}")
print(f"h2o version {h2o.__version__}")

System platform linux
Python version 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
-------------------------------
Pandas version 2.2.2
Numpy version 1.26.4
Matplotlib version 3.10.0
h2o version 3.46.0.6


In [6]:
# Initialise h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.26" 2025-01-21; OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp4h_mt064
  JVM stdout: /tmp/tmp4h_mt064/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp4h_mt064/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,3 months and 7 days
H2O_cluster_name:,H2O_from_python_unknownUser_bmhxux
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


# Data

In [4]:
df = pd.read_csv("height_prediction.csv")
df.head()

,age,gender,mean_ulna,mean_height
0,51,Male,29.0,164.3
1,53,Male,29.0,168.1
2,69,Female,24.0,161.0
3,57,Male,29.0,164.1
4,66,Male,31.0,171.0


In [7]:
df.describe()

,age,mean_ulna,mean_height
count,474.000000,474.000000,474.000000
mean,50.647679,27.883017,162.644937
std,17.298044,1.976854,8.297137
min,18.000000,23.200000,138.900000
25%,36.250000,26.500000,157.000000
50%,53.000000,28.000000,161.575000
75%,65.000000,29.000000,167.775000
max,90.000000,34.500000,190.000000


In [8]:
# Specify target
y = "mean_height"

In [11]:
# Convert dataframe into H2O frame
df = h2o.H2OFrame(df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
# Run AutoML on full dataset
aml = H2OAutoML(max_runtime_secs=300, seed = 1, project_name="Height_Prediction")
aml.train(y = y, training_frame = df)

H2OTypeError: 'training_frame' must be a valid H2OFrame!